<a href="https://colab.research.google.com/github/dockhs1313/19-lab/blob/master/LSTM_2%EA%B0%9C%EC%9D%98_%EC%B8%B5_%EC%8C%93%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:

    %tensorflow_version 2.x  # %tensorflow_version only exists in Colab

except Exception:

    pass

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.x  # %tensorflow_version only exists in Colab`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
tf.__version__

'2.0.0'

## 텐서플로로 LSTM 순환 신경망 만들기

#### 비교를 위한 기존 LSTM 신경망 구축하기

책에서와 같이 LSTM 순환 신경망을 구현해 본다.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model_lstm = Sequential()

model_lstm.add(Embedding(1000,32))
model_lstm.add(LSTM(8))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000     
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 1312      
_________________________________________________________________
dense (Dense)                (None, 1)                 9         
Total params: 33,321
Trainable params: 33,321
Non-trainable params: 0
_________________________________________________________________


In [5]:
from tensorflow.keras.datasets import imdb
(x_train_all, y_train_all), (x_test, y_test)=imdb.load_data(skip_top=20, num_words=1000)
# 앞과 마찬가지로 단어 1000개

for i in range(len(x_train_all)):
  x_train_all[i]=[w for w in x_train_all[i] if w>2]

import numpy as np
np.random.seed(42)
random_index=np.random.permutation(25000)

x_train=x_train_all[random_index[:20000]]
y_train=y_train_all[random_index[:20000]]
x_val=x_train_all[random_index[20000:]]
y_val=y_train_all[random_index[20000:]]

17465344/17464789 [==============================] - 0s 0us/step


LSTM을 한층만 쌓고 10번의 에포크 동안 Adam 옵티마이저를 사용한다.

In [0]:
from tensorflow.keras.preprocessing import sequence

maxlen=100
x_train_seq = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(x_val, maxlen=maxlen)

In [7]:
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model_lstm.fit(x_train_seq, y_train, epochs=10, batch_size=32, validation_data=(x_val_seq, y_val))

loss, accuracy = model_lstm.evaluate(x_val_seq, y_val, verbose=0)
print(accuracy)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 29s 1ms/sample - loss: 0.4576 - accuracy: 0.7964 - val_loss: 0.3788 - val_accuracy: 0.8380
Epoch 2/10
20000/20000 [==============================] - 26s 1ms/sample - loss: 0.3365 - accuracy: 0.8593 - val_loss: 0.3754 - val_accuracy: 0.8338
Epoch 3/10
20000/20000 [==============================] - 26s 1ms/sample - loss: 0.3092 - accuracy: 0.8712 - val_loss: 0.3662 - val_accuracy: 0.8430
Epoch 4/10
20000/20000 [==============================] - 27s 1ms/sample - loss: 0.2922 - accuracy: 0.8781 - val_loss: 0.3679 - val_accuracy: 0.8424
Epoch 5/10
20000/20000 [==============================] - 28s 1ms/sample - loss: 0.2758 - accuracy: 0.8853 - val_loss: 0.3727 - val_accuracy: 0.8420
Epoch 6/10
20000/20000 [==============================] - 27s 1ms/sample - loss: 0.2616 - accuracy: 0.8902 - val_loss: 0.3901 - val_accuracy: 0.8424
Epoch 7/10
20000/20000 [==============================] -

LSTM 한층으로 구현했을 때에는 83.24% 정확도가 나온다.

### LSTM 2개의 층으로 비교하기

return_sequences로 2개의 층을 구현한다.

LSTM 레이어는 return_sequences 인자에 따라 마지막 시퀀스에서 한 번만 출력할 수 있고 각 시퀀스에서 출력을 할 수 있다. many to many 문제를 풀거나 LSTM 레이어를 여러개로 쌓아올릴 때는 return_sequence=True 옵션을 사용한다. 왼쪽은 return_sequences=False일 때, 오른쪽은 return_sequence=True일 때의 형상이다.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model_lstm = Sequential()
model_lstm.add(Embedding(1000,32))
model_lstm.add(LSTM(8, return_sequences=True))
model_lstm.add(LSTM(4,return_sequences=False))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000     
_________________________________________________________________
lstm (LSTM)                  (None, None, 8)           1312      
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 33,525
Trainable params: 33,525
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.datasets import imdb
(x_train_all, y_train_all), (x_test, y_test)=imdb.load_data(skip_top=20, num_words=1000)
# 앞과 마찬가지로 단어 1000개

for i in range(len(x_train_all)):
  x_train_all[i]=[w for w in x_train_all[i] if w>2]

import numpy as np
np.random.seed(42)
random_index=np.random.permutation(25000)

x_train=x_train_all[random_index[:20000]]
y_train=y_train_all[random_index[:20000]]
x_val=x_train_all[random_index[20000:]]
y_val=y_train_all[random_index[20000:]]

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
from tensorflow.keras.preprocessing import sequence

maxlen=100
x_train_seq = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(x_val, maxlen=maxlen)

In [6]:
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model_lstm.fit(x_train_seq, y_train, epochs=10, batch_size=32, validation_data=(x_val_seq, y_val))

loss, accuracy = model_lstm.evaluate(x_val_seq, y_val, verbose=0)
print(accuracy)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 49s 2ms/sample - loss: 0.4562 - accuracy: 0.7885 - val_loss: 0.3835 - val_accuracy: 0.8310
Epoch 2/10
20000/20000 [==============================] - 43s 2ms/sample - loss: 0.3434 - accuracy: 0.8559 - val_loss: 0.3588 - val_accuracy: 0.8452
Epoch 3/10
20000/20000 [==============================] - 45s 2ms/sample - loss: 0.3175 - accuracy: 0.8673 - val_loss: 0.3553 - val_accuracy: 0.8480
Epoch 4/10
20000/20000 [==============================] - 44s 2ms/sample - loss: 0.3003 - accuracy: 0.8751 - val_loss: 0.3535 - val_accuracy: 0.8468
Epoch 5/10
20000/20000 [==============================] - 44s 2ms/sample - loss: 0.2855 - accuracy: 0.8801 - val_loss: 0.3704 - val_accuracy: 0.8404
Epoch 6/10
20000/20000 [==============================] - 45s 2ms/sample - loss: 0.2687 - accuracy: 0.8875 - val_loss: 0.3829 - val_accuracy: 0.8432
Epoch 7/10
20000/20000 [==============================] -

LSTM 한개의 층(83.24%)과 비교하여 두개의 층(83.68%)에서는 약 0.44% 정도 증가함을 볼 수 있다. 첫번째 실행했을때 83.34%로 나왔다가 한번 더 실행해 봤더니 83.68%의 정확도가 나왔다.